In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
#from util import get_data, plot_data
%matplotlib inline



# """MLT: Utility code."""

# import os
# import pandas as pd
# import matplotlib.pyplot as plt

def symbol_to_path(symbol, base_dir="data"):
    """Return CSV file path given ticker symbol."""
    return os.path.join(base_dir, "data/{}.csv".format(str(symbol)))


def get_data(symbols, dates):
    """Read stock data (adjusted close) for given symbols from CSV files."""
    df = pd.DataFrame(index=dates)
    
    if 'SPY' not in symbols:  # add SPY for reference, if absent
        symbols.insert(0, 'SPY')

    for symbol in symbols:
        # TODO: Read and join data for each symbol
        df_temp = pd.read_csv("data/{}.csv".format(symbol), index_col="Date",parse_dates=True,usecols=['Date',
                                                                'Adj Close'],na_values=['nan'])
        
        
        df_temp = df_temp.rename(columns = {'Adj Close' : symbol})
        df = df.join(df_temp)
        df = df.dropna()

    return df

def plot_data(df, title="Stock prices", xlabel="Date", ylabel="Price"):
    """Plot stock prices with a custom title and meaningful axis labels."""
    ax = df.plot(title=title, fontsize=12)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()


def compute_daily_returns(df):
    """Compute and return the daily return values."""
    # Note: Returned DataFrame must have the same number of rows
    #return df.pct_change(1)
    daily_returns = df.copy() # copy df to match size and cloumn
    daily_returns[1:] = df[:-1].values / df[1:] - 1
    daily_returns.ix[0,:] = 0 # set row 0 to 0
    return daily_returns





def compute_daily_returns(df):
    """Compute and return the daily return values."""
    daily_returns = df.copy()
    daily_returns[1:] = (df[1:] / df[:-1].values) - 1
    return daily_returns[1:]

def compute_portfolio_stats(prices, allocs=[0.1,0.2,0.3,0.4], rfr = 0.0, sf = 252.0):
    normed = prices / prices.ix[0]
    alloced = normed * allocs
    port_vals = alloced.sum(axis = 1)
    daily_rets = compute_daily_returns(port_vals)
    cr = port_vals[-1] / port_vals[0] - 1
    adr = daily_rets.mean()
    sddr = daily_rets.std()
    sr = (daily_rets - rfr).mean() / (daily_rets).std() * np.sqrt(sf)
    
    return cr, adr, sddr, sr

def assess_portfolio(sd = dt.datetime(2008,1,1), ed = dt.datetime(2009,1,1), \
    syms = ['GOOG','AAPL','GLD','XOM'], \
    allocs=[0.1,0.2,0.3,0.4], \
    sv=1000000, rfr=0.0, sf=252.0, \
    gen_plot=False):

    # Read in adjusted closing prices for given symbols, date range
    dates = pd.date_range(sd, ed)
    prices_all = get_data(syms, dates)  # automatically adds SPY
    prices = prices_all[syms]  # only portfolio symbols
    prices_SPY = prices_all['SPY']  # only SPY, for comparison later

    # Get daily portfolio value
    
    prices_SPY = prices_SPY / prices_SPY[0]    
    normed = prices / prices.ix[0]
    alloced = normed * allocs
    pos_vals = alloced * sv
    port_vals = pos_vals.sum(axis = 1)

    # Get portfolio statistics (note: std_daily_ret = volatility)
    cr, adr, sddr, sr = compute_portfolio_stats(prices, allocs, rfr, sf)

    # Compare daily portfolio value with SPY using a normalized plot
    if gen_plot:
        # add code to plot here
        df_temp = pd.concat([port_vals/sv, prices_SPY], keys=['Portfolio', 'SPY'], axis=1)
        plot_data(df_temp)
        pass

    # Add code here to properly compute end value
    ev = sv * (1 + cr)
    return cr, adr, sddr, sr, ev

sd = dt.datetime(2010,1,1)
ed = dt.datetime(2010,12,31)
syms =  ['GOOG','AAPL','GLD','XOM']
allocs=[0.2,0.3,0.4,0.1]

cr, adr, sddr, sr, ev = assess_portfolio(sd, ed, syms, allocs, gen_plot=True)
print("Start Date: ", sd)
print("End Date: ", ed)
print("Symbols:", syms)
print("Allocations: ", allocs)
print("Sharpe Ratio: ", sr)
print("Volatility (stdev of daily returns): ", sddr)
print("Average Daily Return: ", adr)
print("Cumulative Return: ", cr)

ValueError: Wrong number of items passed 4, placement implies 5